In [30]:
import numpy as np 
import pandas as pd
import math
eps = np.finfo(float).eps

In [31]:
data  = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

In [32]:
df = pd.DataFrame(data, columns = ['Taste','Temperature','Texture','Eat'])
df

,Taste,Temperature,Texture,Eat
0,Salty,Hot,Soft,No
1,Spicy,Hot,Soft,No
2,Spicy,Hot,Hard,Yes
3,Spicy,Cold,Hard,No
4,Spicy,Hot,Hard,Yes
5,Sweet,Cold,Soft,Yes
6,Salty,Cold,Soft,No
7,Sweet,Hot,Soft,Yes
8,Spicy,Cold,Soft,Yes
9,Salty,Hot,Hard,Yes


In [33]:
entr_target = 0.0
results = df.Eat.unique() 
targets = df.Eat.unique()
for result in results:
    probability = df.Eat.value_counts()[result]/len(df)
    entr_target += -probability*math.log(probability,2)
entr_target

0.9709505944546686

In [34]:
attributes = ['Taste','Temperature','Texture']
for attribute in attributes:
    entropy_attribute = 0
    classes = df[attribute].unique()
    print(classes)

['Salty' 'Spicy' 'Sweet']
['Hot' 'Cold']
['Soft' 'Hard']


In [35]:
entropy_attributes = {}
gain_attributes = {}
for attribute in attributes:
    entropy_att=0
    classes = df[attribute].unique()
    
    for each_class in classes:
        entropy_each_class = 0
        
        for target in targets:
            numerator = len(df[attribute][df[attribute]==each_class][df.Eat ==target])
            denominator = len(df[attribute][df[attribute]==each_class])
            prob = numerator/(denominator+eps)
            entropy_each_class += abs(-prob*math.log((prob+eps),2))
            
        prob_outer = denominator/len(df)
        entropy_att += abs(-prob_outer*entropy_each_class)
        
    entropy_attributes[attribute] = entropy_att
    gain_attributes[attribute] = entr_target- entropy_att
print(entropy_attributes)
print(gain_attributes) 

{'Taste': 0.7609640474436808, 'Temperature': 0.950977500432693, 'Texture': 0.9245112497836524}
{'Taste': 0.20998654701098773, 'Temperature': 0.019973094021975557, 'Texture': 0.04643934467101618}


In [36]:
def entropy_calc(df):
    Class = df.keys()[-1]
    entropy = 0
    values_attributes = df[Class].unique()
    for value_attribute in values_attributes:
        probability = df.Eat.value_counts()[result]/len(df)
        entropy += -probability*np.log2(probability)
    return entropy

In [37]:
def entropy_calc_attribute(df,attribute):
    Class = df.keys()[-1]
    targets = df[Class].unique()
    values = df[attribute].unique()
    entropy2 = 0
    for value in values:
        entropy = 0
        for target in targets:
            numerator = len(df[attribute][df[attribute]==value][df[Class] ==target])
            denominator = len(df[attribute][df[attribute]==value])
            prob = numerator/(denominator+eps)
            entropy += -prob*math.log(prob,2)
        prob2 = denominator/len(df)
        entropy2 += -prob2*entropy
    return abs(entropy2)

In [38]:
def find_next_attribute(df):
    entropy_attributes = []
    info_gain = []
    for key in df.keys()[:-1]:
        entropy_attributes.append(entropy_calc_attribute(df,key))
        info_gain.append(entropy_calc(df)-entropy_calc_attribute(df,key))
    return df.keys()[:-1][np.argmax(info_gain)]

In [39]:
def remaining_dataset(df, node, value):
    return df[df[node]==value].reset_index(drop=True)

In [40]:
def build_final_tree(df, tree=None):
    Class = df.keys()[-1]
    node = find_next_attribute(df)
    attribute_value = np.unique(df[node])
    if tree is None:
        tree = {}
        tree[node] = {}
    for value in attribute_value:
        remaining_tree = remaining_dataset(df,node,value)
        clValue,counts = np.unique(remaining_tree['Eat'],return_counts=True)
        
        if len(counts)==1:
            tree[node][value] = clValue[0]
        else: 
            tree[node][value] = build_final_tree(remaining_tree)


In [41]:
tree = build_final_tree(df)
print(tree)

ValueError: math domain error